# IMDB영화 평점 예측

# 라이브러리

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer # text -> vector

from sklearn.preprocessing import LabelEncoder # 평점 -> label로 변환

import numpy as np

# 모델 정의

In [2]:
class myRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(myRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:,-1,:])
        return out


# 데이터 로딩

In [3]:
from google.colab import files
files.upload()

Saving IMDB_reviews_binary_suicide_squad.csv to IMDB_reviews_binary_suicide_squad.csv


{'IMDB_reviews_binary_suicide_squad.csv': b',review,rating\r\n0,Bad Company,1\r\n1,I am so-so sad :(,0\r\n2,"Great characters, not so great story",1\r\n3,"Apparently it\'s good to be bad, should be DC\'s tagline",0\r\n4,"Despite what many critics have to say, this is a very good movie",1\r\n5,Looking forward to more Harley and Deadshot!,1\r\n6,Disappointing and messy :(,0\r\n7,David Ayer brings something new for the DC Cinematic Universe.,1\r\n8,actually OK with lots of annoying little problems,1\r\n9,Corny and predictable,0\r\n10,Walk to the building the movie,1\r\n11,It\'s not as good as your expect nor as bad as you heard,0\r\n12,Great acting with some funny jokes and good characters even if a bit of a mess,1\r\n13,A very poor attempt in making a Suicide Squad film,0\r\n14,The Comic vs. The Movie,1\r\n15,Mediocre,1\r\n16,So Bad Even Shills Can\'t Ensure The Truth Won\'t Get Out,0\r\n17,"Don\'t listen to the so called ""critics"" , the movie is fun",1\r\n18,"Not a masterpiece, but a 

In [22]:
data = pd.read_csv("/content/drive/MyDrive/4-1 빅데이터와 딥러닝/IMDB_reviews_binary_suicide_squad.csv", encoding='unicode_escape')

reviews = data['review'].tolist()
rating = data['rating'].tolist()

print(len(reviews), len(rating))

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews).toarray()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(rating)

2471 2471


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=42)

In [26]:
class ReviewDataset(Dataset):
    def __init__(self, x, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.int64)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = ReviewDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size = 2, shuffle=True)

In [27]:
train_dataset.__len__()

1729

# 초기화

In [20]:
input_size = X.shape[1]
hidden_size = 16
output_size = 2
learning_rate = 0.01
num_epochs = 100

model = myRNN(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# 훈련

In [21]:
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1) # max는 두개 값을 리턴하는데 첫번째는 max값 두번째는 인덱스값

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    if (epoch+1) % 2 == 0:
        print(f'Epoch[{epoch+1}/{num_epochs}], Loss: {loss.item()}')
    if (loss.item() < 0.00001):
        break

IndexError: index 1856 is out of bounds for dimension 0 with size 1729